ref: https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On-Second-Edition/blob/master/Chapter04/01_cartpole.py

In [1]:
import gym
from collections import namedtuple
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from tensorboardX import SummaryWriter

In [2]:
# vanilla NN
class Net(nn.Module):
    # probability distribution over actions (should include softmax as last layer)
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
            # should include softmax layer but the `nn.CrossEntropyLoss()` does it for us
        )

    def forward(self, x):
        return self.net(x)

In [3]:
# represent 1 step of the agent in the environment, will store observation and taken action
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])
# store undiscounted reward and a collection of steps as `EpisodeStep`
Episode = namedtuple('Episode', field_names=['reward', 'steps'])

def iterate_batches(env, net, batch_size):
    """ generate batches of episodes based on selected `batch_size` """

    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)

    while True:
        # convert observation to torch tensor
        obs_v = torch.FloatTensor([obs])
        
        # forward pass through the network
        pred_logits = net(obs_v)
        # apply softmax to get probability distribution over actions
        act_probs_v = sm(pred_logits)
        
        # select action based on probability distribution
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        
        # take action in the environment and observe imidiate reward and next state
        next_obs, reward, done, _ = env.step(action)
        
        # update cummulative episode reward
        episode_reward += reward
        
        # store step information
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        
        if done:
            # store episode information
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            
            # reset the environment
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            
            # yield the generated batch if it's full
            if len(batch) == batch_size:
                yield batch
                batch = []

        # update observation
        obs = next_obs

In [4]:
def filter_batch(batch, percentile):
    """ filter only the elite episodes based on the percentile """

    # calculate batch average reward and percentile threshold
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    # select elite episodes based on `reward_bound` threshold
    train_obs = []
    train_act = []
    for episode_reward, steps in batch:
        if episode_reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))

    # preprocess elite episodes (observations, actions) to torch tensors
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)

    return train_obs_v, train_act_v, reward_bound, reward_mean

In [8]:
# create CartPole environment
env = gym.make("CartPole-v0")
# env = gym.wrappers.Monitor(env, directory="mon", force=True)  # unconmment to record video

obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

In [9]:
HIDDEN_SIZE = 128

# define network, loss function and optimizer
net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()  # combination of softmax and cross-entropy calculation (required logits as input)
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

In [10]:
BATCH_SIZE = 16
PERCENTILE = 70

# iterate over batches of episodes and train the network
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    # filter elite episodes from the generated batch
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    
    # train the network
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    
    if (iter_no+1) % 10 == 0:
        print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (iter_no+1, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    
    # stop training if the average reward reached 200 (environment maximum score), considered as solved
    if reward_m > 199:
        print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (iter_no+1, loss_v.item(), reward_m, reward_b))
        print("Solved!")
        break

writer.close()

/var/folders/rd/xnkdmhqx0c19zv0s0z9zcc9c0000gn/T/ipykernel_9927/1877622033.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  obs_v = torch.FloatTensor([obs])


10: loss=0.639, reward_mean=49.2, rw_bound=49.0
20: loss=0.598, reward_mean=91.4, rw_bound=99.5
30: loss=0.556, reward_mean=125.3, rw_bound=144.5
40: loss=0.530, reward_mean=198.5, rw_bound=200.0
49: loss=0.524, reward_mean=200.0, rw_bound=200.0
Solved!
